In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/B2B_Order_Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Supplier_ID             1002 non-null   object 
 1   Order_Date              1002 non-null   object 
 2   Shipping_Date           1002 non-null   object 
 3   Expected_Delivery_Date  1002 non-null   object 
 4   Product_Code            1002 non-null   object 
 5   Product_Name            1002 non-null   object 
 6   Per_Unit_Cost           1001 non-null   float64
 7   Quantity                1002 non-null   int64  
 8   Total_Cost              999 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 70.6+ KB


(  Supplier_ID  Order_Date Shipping_Date Expected_Delivery_Date Product_Code  \
 0        SUP1  2025-03-24    2025-03-30             2025-03-31         P001   
 1        SUP1  2025-04-22    2025-04-28             2025-05-04         P003   
 2        SUP1  2025-07-22    2025-07-29             2025-07-30         P003   
 3        SUP1  2024-10-04    2024-10-08             2024-10-18         P005   
 4        SUP1  2024-10-19    2024-11-02             2024-11-08         P005   
 
        Product_Name  Per_Unit_Cost  Quantity  Total_Cost  
 0         Inverters          38.49        85     3271.65  
 1  Onboard Chargers         113.46       142    16111.32  
 2  Onboard Chargers         119.66       140    16752.40  
 3            Struts         349.00       184    64216.00  
 4            Struts         372.14        48    17862.72  ,
 None)

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/B2B_Order_Data.csv'
data = pd.read_csv(file_path)

# Remove duplicate rows
data = data.drop_duplicates()

# Remove rows with missing values
data = data.dropna()

# Save the cleaned dataset to a new CSV file
cleaned_file_path = '/content/Cleaned_B2B_Order_Data.csv'
data.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to {cleaned_file_path}")


Cleaned dataset saved to /content/Cleaned_B2B_Order_Data.csv


In [5]:
import pandas as pd

# Load the cleaned dataset
cleaned_file_path = '/content/Cleaned_B2B_Order_Data.csv'
data = pd.read_csv(cleaned_file_path)

# Convert date columns to datetime
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
data['Expected_Delivery_Date'] = pd.to_datetime(data['Expected_Delivery_Date'])

# Calculate lead time to delivery
data['lead_time_to_delivery'] = (data['Expected_Delivery_Date'] - data['Order_Date']).dt.days

# Save the updated dataset to a new CSV file
updated_file_path = '/content/Updated_B2B_Order_Data.csv'
data.to_csv(updated_file_path, index=False)

print(f"Updated dataset with lead time to delivery saved to {updated_file_path}")


Updated dataset with lead time to delivery saved to /content/Updated_B2B_Order_Data.csv


In [6]:
import pandas as pd

def calculate_average_lead_time(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # Convert date columns to datetime
    data['Order_Date'] = pd.to_datetime(data['Order_Date'])
    data['Expected_Delivery_Date'] = pd.to_datetime(data['Expected_Delivery_Date'])

    # Calculate lead time to delivery
    data['lead_time_to_delivery'] = (data['Expected_Delivery_Date'] - data['Order_Date']).dt.days

    # Calculate the average lead time for each supplier
    average_lead_time = data.groupby('Supplier_ID')['lead_time_to_delivery'].mean().reset_index()

    return average_lead_time

# Example usage
file_path = '/content/Cleaned_B2B_Order_Data.csv'
average_lead_time_df = calculate_average_lead_time(file_path)
print(average_lead_time_df)


  Supplier_ID  lead_time_to_delivery
0        SUP1              13.718935
1        SUP2              13.079646
2        SUP3              13.332288


In [8]:
import numpy as np
import pandas as pd

# Parameters
num_simulations = 100
num_days = 180
mean_demand = 50
std_dev_demand = 10
holding_cost_per_unit_per_day = 10
shortage_cost_per_unit_per_day = 50
delivery_times = {
    'Supplier 1': 13.718935,
    'Supplier 2': 13.079646,
    'Supplier 3': 13.332288
}

# Function to run a single simulation
def run_simulation(delivery_time):
    total_holding_cost = 0
    total_shortage_cost = 0
    inventory_level = 0

    daily_demand = np.random.normal(mean_demand, std_dev_demand, num_days)

    for day in range(num_days):
        demand = daily_demand[day]
        inventory_level -= demand

        # If inventory is negative, calculate shortage cost
        if inventory_level < 0:
            total_shortage_cost += abs(inventory_level) * shortage_cost_per_unit_per_day
            inventory_level = 0  # Reset inventory level to 0 after shortage

        # Calculate holding cost
        total_holding_cost += inventory_level * holding_cost_per_unit_per_day

        # Simulate delivery after the specified lead time
        if day % delivery_time == 0:
            inventory_level += mean_demand * delivery_time  # Replenish inventory

    return total_holding_cost, total_shortage_cost

# Running simulations for each supplier
results = {'Supplier': [], 'Simulation': [], 'Total_Holding_Cost': [], 'Total_Shortage_Cost': []}

for supplier, delivery_time in delivery_times.items():
    for sim in range(num_simulations):
        holding_cost, shortage_cost = run_simulation(delivery_time)
        results['Supplier'].append(supplier)
        results['Simulation'].append(sim + 1)
        results['Total_Holding_Cost'].append(holding_cost)
        results['Total_Shortage_Cost'].append(shortage_cost)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display results
print(results_df.head())

# Save results to a CSV file
results_df.to_csv('/content/Inventory_Management_Simulation_Results.csv', index=False)


     Supplier  Simulation  Total_Holding_Cost  Total_Shortage_Cost
0  Supplier 1           1        44386.021217        421449.922015
1  Supplier 1           2        53107.827872        412599.245157
2  Supplier 1           3        37797.886815        424503.212769
3  Supplier 1           4        44123.084737        415889.045539
4  Supplier 1           5        41898.619623        418354.215870


In [11]:
import numpy as np
import pandas as pd

# Parameters
num_simulations = 100
num_days = 180
mean_demand = 50
std_dev_demand = 10
holding_cost_per_unit_per_day = 10
shortage_cost_per_unit_per_day = 50
delivery_times = {
    'Supplier 1': 13.718935,
    'Supplier 2': 13.079646,
    'Supplier 3': 13.332288
}

# Function to run a single simulation
def run_simulation(delivery_time):
    total_holding_cost = 0
    total_shortage_cost = 0
    inventory_level = 0

    daily_demand = np.random.normal(mean_demand, std_dev_demand, num_days)

    for day in range(num_days):
        demand = daily_demand[day]
        inventory_level -= demand

        # If inventory is negative, calculate shortage cost
        if inventory_level < 0:
            total_shortage_cost += abs(inventory_level) * shortage_cost_per_unit_per_day
            inventory_level = 0  # Reset inventory level to 0 after shortage

        # Calculate holding cost
        total_holding_cost += inventory_level * holding_cost_per_unit_per_day

        # Simulate delivery after the specified lead time
        if day % delivery_time == 0:
            inventory_level += mean_demand * delivery_time  # Replenish inventory

    return total_holding_cost, total_shortage_cost

# Running simulations for each supplier
def run_simulations_and_calculate_average_costs():
    results = {'Supplier': [], 'Simulation': [], 'Total_Holding_Cost': [], 'Total_Shortage_Cost': []}

    for supplier, delivery_time in delivery_times.items():
        for sim in range(num_simulations):
            holding_cost, shortage_cost = run_simulation(delivery_time)
            results['Supplier'].append(supplier)
            results['Simulation'].append(sim + 1)
            results['Total_Holding_Cost'].append(holding_cost)
            results['Total_Shortage_Cost'].append(shortage_cost)

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Calculate average total inventory cost by supplier
    results_df['Total_Inventory_Cost'] = results_df['Total_Holding_Cost'] + results_df['Total_Shortage_Cost']
    average_costs = results_df.groupby('Supplier')['Total_Inventory_Cost'].mean().reset_index()

    return average_costs

# Example usage
average_costs_df = run_simulations_and_calculate_average_costs()
print(average_costs_df)

# Save average costs to a CSV file
average_costs_df.to_csv('/content/Average_Inventory_Costs_By_Supplier.csv', index=False)


     Supplier  Total_Inventory_Cost
0  Supplier 1         460282.112588
1  Supplier 2         456729.291583
2  Supplier 3         458543.873010
